In [15]:
import pandas as pd
import numpy as np
import streamlit as st
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import load_model,save_model
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import datetime


In [3]:
data = pd.read_csv('Churn_Modelling.csv')
# Preprocessing the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [5]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


In [7]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))


In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
data = pd.concat([data, geo_encoded_df], axis=1)
data = data.drop(['Geography'], axis=1)

In [10]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [11]:
X = data.drop(['EstimatedSalary'], axis=1)
y = data['EstimatedSalary']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geo, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

#### ANN Regression Problem Statement

In [16]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)
    ]
)

model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_absolute_error', metrics=['mean_absolute_error'])


/Users/vd045211/my-projects/aiml/Complete_AI_ML_DS_Krish_Naik/code/end-end-ann/myvenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
log_dir = 'sal_regression_logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 79679.9219 - mean_absolute_error: 79679.9219 - val_loss: 50915.3164 - val_mean_absolute_error: 50915.3164
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50209.3125 - mean_absolute_error: 50209.3125 - val_loss: 50322.5664 - val_mean_absolute_error: 50322.5664
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49856.5352 - mean_absolute_error: 49856.5352 - val_loss: 50288.8359 - val_mean_absolute_error: 50288.8359
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49721.9961 - mean_absolute_error: 49721.9961 - val_loss: 50399.2812 - val_mean_absolute_error: 50399.2812
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49606.8828 - mean_absolute_error: 49606.8828 - val_loss: 50281.7070 - val_mean_absolute_error: 50281.7070
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 49569.7617 - mean_absolute_error: 49569.7617 - val_loss: 50354.4922 - val_mean_absolute_error: 50354.492

In [19]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50281.7070 - mean_absolute_error: 50281.7070
Test Loss: 50281.70703125, Test MAE: 50281.70703125


In [20]:
model.save('salary_regression_model.h5')